In [7]:
import pickle
import pandas as pd
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import matplotlib.dates as mdates
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional
from keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from tensorflow.keras.models import load_model
import pickle
import openpyxl
import tensorflow
from sklearn.metrics import mean_squared_error, r2_score
import plotly.express as px
import plotly.graph_objects as go
# Load the model from the pickle file
with open('SVR.pkl', 'rb') as f:
    model = pickle.load(f)
    



In [8]:
print(model)

SVR(C=1000.0, gamma=0.1)


In [9]:
rawdataset = pd.read_excel('grocery_price.xlsx', index_col=None)
dataset = rawdataset.T
dataset.columns = dataset.iloc[0]
dataset = dataset.drop(dataset.index[0])
dataset['Average'] = round(dataset.mean(axis=1),2)

original = dataset.drop(dataset.columns[0:3], axis=1)
original.reset_index(inplace=True)
original = original.rename(columns={'index':'Date'})
original = original.values[0:963]


dataset = dataset.drop(dataset.columns[0:3], axis=1)
dataset = dataset.values

sc = MinMaxScaler(feature_range=(0,1))
scaled_dataset = sc.fit_transform(dataset)

def create_sequences(data, seq_length=60):
  x=[]
  y=[]
  for i in range(seq_length, len(data)):
      x.append(data[i-seq_length:i,0])
      y.append(data[i,0])
  return np.array(x), np.array(y)
x_test, y_test = create_sequences(scaled_dataset)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [10]:

# # Predict the stock price
x_pred = x_test.reshape(x_test.shape[0], x_test.shape[1])
predicted_price = model.predict(x_pred)
predicted_price = sc.inverse_transform(predicted_price.reshape(-1,1))
index = np.arange(len(predicted_price))
predicted_price_acc = np.column_stack((index, predicted_price))
original_acc = np.column_stack((index, original[:,1]))

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=original[:,0], y=original[:,1], mode='lines', name='Grocery Price', line=dict(color='red')))
fig.add_trace(go.Scatter(x=original[:,0], y=predicted_price[:,0], mode='lines', name='Predicted Grocery Price', line=dict(color='green')))



In [12]:
mse = mean_squared_error(original_acc, predicted_price_acc)
r2 = r2_score(original_acc, predicted_price_acc)
print(mse)
print(r2)

0.16384542579778508
0.8201729128568247
